This is merely a sample of data from the interscholastic Track & Field and Cross Country statistics website Athletic.Net.

In order to make my examples clear and easy to follow, I have decided to play around with just a few rows (in particular, stats for my brother and I; me back from when I was running XC in high school) taken from Althletic.Net statistics I parsed the other day using python's Beautiful Soup module.

I will likely include a link to a more comprehensive web-scraping and statistical analysis project using some of this data, but as of the time of this writing, I have yet to decide exactly what I want to do with it.

I want to get this short SQL/sqlite/sqlalchemy sample uploaded first, because of how in-demand basic db-interaction/querying skills seem to be when it comes to entry-level positions.

In [1]:
import sqlite3
from sqlalchemy import create_engine, MetaData

# Create connection to local database file
engine = create_engine('sqlite:///xc_profiles.db')
connection = engine.connect()
print(engine)
metadata = MetaData(engine)
for tbl in reversed(metadata.sorted_tables):
    engine.execute(tbl.delete())

metadata.drop_all()
print(engine.table_names())
print(connection)

Engine(sqlite:///xc_profiles.db)
[]


In [2]:
from sqlalchemy import Table, Column, String, Integer, Float, Boolean

metadata.drop_all()
metadata.clear()
print(metadata.reflect())

# NOTE: for reference, and those unfamiliar with sqlalchemy syntax,
# I will include the equivalent SQL statement as a comment above each operation.


print(engine.table_names())
# CREATE TABLE example: """CREATE TABLE stats2 (Name VARCHAR, Event VARCHAR, Year INTEGER, ...etc)"""
stats2 = Table('stats2', metadata,
             Column('Name', String(255)),
             Column('Event', String(255)),
             Column('Year', Integer()),
             Column('Grade', String(255)),
             Column('Time', String(255)), extend_existing=True)

# Use metadata to create the stats2 table defined above
metadata.create_all(engine)

# Confrim the tables we have to work with. stats2 should be present.
print(engine.table_names())

None
[]
['stats2']


In [3]:
# Print stats2 table details
print(repr(stats2))

Table('stats2', MetaData(bind=Engine(sqlite:///xc_profiles.db)), Column('Name', String(length=255), table=<stats2>), Column('Event', String(length=255), table=<stats2>), Column('Year', Integer(), table=<stats2>), Column('Grade', String(length=255), table=<stats2>), Column('Time', String(length=255), table=<stats2>), schema=None)


In [4]:
from sqlalchemy import insert, select

# As mentioned in the introduction, for simplicity and reference
# I am just working with a few rows from my athletic.net data:
stats_list = [
    {'Name': 'Wesley Warriner', 'Event': '3,000 Meters', 'Year': 2012, 'Grade': '10th Grade', 'Time': '13:05.0 PR'},
    {'Name': 'Wesley Warriner', 'Event': '3,000 Meters', 'Year': 2014, 'Grade': '12th Grade', 'Time': '14:12.0'},
    {'Name': 'Wesley Warriner', 'Event': '5,000 Meters', 'Year': 2011, 'Grade': '9th Grade', 'Time': '18:57.0'},
    {'Name': 'Wesley Warriner', 'Event': '5,000 Meters', 'Year': 2012, 'Grade': '10th Grade', 'Time': '17:48.1 PR'},
    {'Name': 'Wesley Warriner', 'Event': '5,000 Meters', 'Year': 2013, 'Grade': '11th Grade', 'Time': '19:30.0'},
    {'Name': 'Wesley Warriner', 'Event': '5,000 Meters', 'Year': 2013, 'Grade': '12th Grade', 'Time': '19:00.0'},    
    {'Name': 'Duncan Warriner', 'Event': '5,000 Meters', 'Year': 2016, 'Grade': '9th Grade', 'Time': '19:07.0'},
    {'Name': 'Duncan Warriner', 'Event': '5,000 Meters', 'Year': 2017, 'Grade': '10th Grade', 'Time': '18:29.6 PR'}
]


# Prepare INSERT statement for stats_list entries into stats2 table: 
#   """INSERT INTO stats2 ("Name", "Event", "Year", "Grade", "Time") VALUES (:Name, :Event, :Year, :Grade, :Time)"""
insert_stmt = insert(stats2)
# btw this is how you can quickly retrieve the string equivalent:
print(str(insert_stmt))

# Execute stmt with the stats_list list of dicts
insert_results = connection.execute(insert_stmt, stats_list)

# Print rowcount to confirm INSERT
print('NumRows:',insert_results.rowcount)

INSERT INTO stats2 ("Name", "Event", "Year", "Grade", "Time") VALUES (?, ?, ?, ?, ?)
NumRows: 8


In [5]:
# SELECT statement: """SELECT * FROM stats2 WHERE Name == 'Duncan Warriner'"""
select_stmt = select([stats2]).where(stats2.columns.Name == 'Duncan Warriner')
select_results = connection.execute(select_stmt).fetchall()
print(select_results)

[('Duncan Warriner', '5,000 Meters', 2016, '9th Grade', '19:07.0'), ('Duncan Warriner', '5,000 Meters', 2017, '10th Grade', '18:29.6 PR')]


In [6]:
from sqlalchemy import and_

# one more simple SELECT statement: """SELECT Time FROM stats2 WHERE Name == 'Wesley Warriner' AND Event == '3,000 Meters'"""
select_stmt = select([stats2.columns.Time]).where(
    and_(
        stats2.columns.Name == 'Wesley Warriner',
        stats2.columns.Event == '3,000 Meters'))
select_results = connection.execute(select_stmt).fetchall()
print(select_results)

[('13:05.0 PR',), ('14:12.0',)]


In [7]:
from sqlalchemy import update

# UPDATE statement example: """UPDATE stats2 SET Grade = '9th Grade' WHERE Name = 'Duncan Warriner' AND Year = 2017"""
update_stmt = update(stats2).values(Grade='9th Grade').where(
    and_(
        stats2.columns.Name == 'Duncan Warriner',
        stats2.columns.Year == 2017)
    )
connection.execute(update_stmt)

# confirmation SELECT statement: """SELECT * FROM stats2 WHERE Grade == '9th Grade'
#   AND Name == 'Duncan Warriner' AND Year == 2017"""
select_stmt = select([stats2.columns.Grade]).where(
    and_(
        stats2.columns.Name == 'Duncan Warriner',
        stats2.columns.Year == 2017
    ))
    
select_results = connection.execute(select_stmt).fetchall()
print(select_results)

[('9th Grade',)]


In [8]:
# And actually let's go ahead and revert what we just updated
update_stmt = update(stats2).values(Grade='10th Grade').where(
    and_(
        stats2.columns.Name == 'Duncan Warriner',
        stats2.columns.Year == 2017)
    )
connection.execute(update_stmt)

# confirmation SELECT statement: """SELECT * FROM stats2 WHERE Grade == '10th Grade'
#   AND Name == 'Duncan Warriner' AND Year == 2017"""
select_stmt = select([stats2.columns.Grade]).where(
    and_(
        stats2.columns.Name == 'Duncan Warriner',
        stats2.columns.Year == 2017
    ))
    
select_results = connection.execute(select_stmt).fetchall()
print(select_results)

[('10th Grade',)]


In [9]:
from sqlalchemy import or_, desc
# ORDER BY example: """SELECT * FROM stats2 WHERE (Name == 'Wesley Warriner' OR Name == 'Duncan Warriner') AND 
#   (Grade == '9th Grade' OR Grade == '10th Grade') ORDER BY 'Year' DESC"""
stmt = select([stats2]).where( 
    and_(
        or_(
            #I know these are currently the only two names, just an example of usage:
            stats2.columns.Name == 'Wesley Warriner',
            stats2.columns.Name == 'Duncan Warriner'),
        or_(
            stats2.columns.Grade == '9th Grade',
            stats2.columns.Grade == '10th Grade')
    ))
ordered_stmt = stmt.order_by(desc(stats2.columns.Year))

ordered_results = connection.execute(ordered_stmt).fetchall()
print(ordered_results)

[('Duncan Warriner', '5,000 Meters', 2017, '10th Grade', '18:29.6 PR'), ('Duncan Warriner', '5,000 Meters', 2016, '9th Grade', '19:07.0'), ('Wesley Warriner', '3,000 Meters', 2012, '10th Grade', '13:05.0 PR'), ('Wesley Warriner', '5,000 Meters', 2012, '10th Grade', '17:48.1 PR'), ('Wesley Warriner', '5,000 Meters', 2011, '9th Grade', '18:57.0')]


In [10]:
# Maybe just a few more quickies.
# We can also include functions, such as COUNT:
from sqlalchemy import func

# """SELECT COUNT(DISTINCT Year) AS count_1 FROM stats2"""
count_distinct_stmt = select([func.count(stats2.columns.Year.distinct())])

year_distinct_count = connection.execute(count_distinct_stmt).scalar()
print('Distinct years:', year_distinct_count, 'years')

Distinct years: 6 years


In [11]:
from sqlalchemy import ForeignKey
# Alright let's bring another table into the mix
#   Note: the following is not Athletic.Net data. I'm just creating and using it here
athletes2 = Table('athletes2', metadata,
             Column('Name', String(255), ForeignKey('stats2.Name'), primary_key=True),
             Column('School', String(255)),
             Column('Graduation_Status', String(255)), extend_existing=True
)

metadata.create_all(engine)
print(engine.table_names())
# Print athletes2 table details
print(repr(athletes2))

['athletes2', 'stats2']
Table('athletes2', MetaData(bind=Engine(sqlite:///xc_profiles.db)), Column('Name', String(length=255), ForeignKey('stats2.Name'), table=<athletes2>, primary_key=True, nullable=False), Column('School', String(length=255), table=<athletes2>), Column('Graduation_Status', String(length=255), table=<athletes2>), schema=None)


In [12]:
athletes_list = [
    {'Name': 'Wesley Warriner', 'School': 'Lake Washington', 'Graduation_Status': 'Yes'},
    {'Name': 'Duncan Warriner', 'School': 'Redmond', 'Graduation_Status': 'No'},
    {'Name': 'Cody Banks', 'School': 'Ballard', 'Graduation_Status': 'No'}
]

insert_stmt = insert(athletes2)
insert_results = connection.execute(insert_stmt, athletes_list)
# Print rowcount to confirm INSERT
print(insert_results.rowcount)

3


In [13]:
# stats2 JOIN athletes2 ON stats2.'Name' = athletes2.'Name'
stmt = select([stats2.columns.Year, athletes2.columns.Graduation_Status])
join_select_stmt = stmt.select_from(stats2.join(athletes2))

print(str(join_select_stmt))

results = connection.execute(join_select_stmt).fetchall()
print(results)

SELECT stats2."Year", athletes2."Graduation_Status" 
FROM stats2 JOIN athletes2 ON stats2."Name" = athletes2."Name"
[(2012, 'Yes'), (2014, 'Yes'), (2011, 'Yes'), (2012, 'Yes'), (2013, 'Yes'), (2013, 'Yes'), (2016, 'No'), (2017, 'No')]
